In [2]:
from langchain_community.document_loaders import PyPDFium2Loader
from pathlib import Path

PDF_PATH = Path("data/justforfun_persian.pdf")
if not PDF_PATH.exists():
    raise FileNotFoundError(f"Couldn't find the PDF at {PDF_PATH.resolve()}")
loader = PyPDFium2Loader(str(PDF_PATH))
pdf_docs = loader.load()


In [3]:
print("Number of pages (loaded Document objects):", len(pdf_docs))


Number of pages (loaded Document objects): 204


In [4]:
page_index = 15
if page_index >= len(pdf_docs):
    raise IndexError(
        f"Document has only {len(pdf_docs)} pages; can't show index {page_index}."
    )
print(pdf_docs[page_index].page_content)


تولد یͷ نرد، بخش دوم صفحۀ ١١
و او مͬ خواست من را هم در این تجربه شریͷ کند. همچنین مͬ خواست من را به ریاضͬ
علاقمند کند.
پس من را روی زانو هایش مͬ نشاند و از من مͬ خواست تا برنامه هایی که با دقت روی کاغذ
نوشته بود را برایش تایپ کنم. مͬ گفت خودش با کامپیوترها راحت نیست. نمͬ دانم آن محاسبات
راجع به چه چیزی بودند و بعید مͬ دانم که آن موقع هیچ درکͬ از کاری که مͬ کردم هم داشته باشم
ولͬ به هرحال آنجا بودم و به او کمͷ مͬ کردم. احتمالا کار از حالتͬ که او خودش به تنهایی
برنامه ها را وارد مͬ کرد، خیلͬ بیشتر طول مͬ کشید. ولͬ کسͬ چه مͬ داند؟ من از همان کودکͬ به
صفحه کلید عادت کرده بودم، چیزی که پدربزرگم هیچ وقت امͺان اش را نداشت. بعد از مدرسه
یا هر موقع دیͽری که مادرم من را پیش پدربزرگم مͬ گذاشت، مشغول همین کار مͬ شدیم.
بعد شروع کردم به خواندن راهنماهای کامپیوتر و وارد کردن برنامه های آماده شده. مثال ها
شامل بازی های ساده ای بودند که خودتان مͬ توانستید آن ها را وارد کنید. اگر همه چیز را درست
تایپ مͬ کردید، یͷ آقایی با گرافیͷ بد، روی صفحه راه مͬ رفت. بعد مͬ توانستید برنامه را عوض
کنید تا آقای راه رون

In [5]:
import unicodedata
from collections import Counter


def collect_char_stats(docs):
    cnt = Counter()
    for d in docs:
        cnt.update(d.page_content)
    return cnt


stats = collect_char_stats(pdf_docs)


def is_expected_char(ch: str) -> bool:
    cp = ord(ch)
    if "0" <= ch <= "9" or "A" <= ch <= "Z" or "a" <= ch <= "z":
        return True
    if ch in " \n\r\t.,;:!?()[]{}«»'\"/\\-–—…،٪%‌":
        return True
    if 0x0600 <= cp <= 0x06FF:
        return True
    if 0x0750 <= cp <= 0x077F:
        return True
    if 0x08A0 <= cp <= 0x08FF:
        return True
    return False


suspects = [
    (c, n, ord(c), unicodedata.name(c, "?"), unicodedata.category(c))
    for c, n in stats.most_common()
    if not is_expected_char(c) and c.strip() != ""
]
print(f"Unique chars: {len(stats)}  |  Suspects: {len(suspects)}")
for c, n, code, name, cat in suspects[:50]:
    print(f"{repr(c)}  U+{code:04X}  {name}  cat={cat}  count={n}")


Unique chars: 166  |  Suspects: 32
'ͬ'  U+036C  COMBINING LATIN SMALL LETTER R  cat=Mn  count=7751
'ͺ'  U+037A  GREEK YPOGEGRAMMENI  cat=Lm  count=1595
'ͷ'  U+0377  GREEK SMALL LETTER PAMPHYLIAN DIGAMMA  cat=Ll  count=1559
'ͽ'  U+037D  GREEK SMALL REVERSED DOTTED LUNATE SIGMA SYMBOL  cat=Ll  count=915
'“'  U+201C  LEFT DOUBLE QUOTATION MARK  cat=Pi  count=348
'”'  U+201D  RIGHT DOUBLE QUOTATION MARK  cat=Pf  count=348
'‐'  U+2010  HYPHEN  cat=Pd  count=236
'\u0379'  U+0379  ?  cat=Cn  count=14
'&'  U+0026  AMPERSAND  cat=Po  count=10
'@'  U+0040  COMMERCIAL AT  cat=Po  count=10
'˼'  U+02FC  MODIFIER LETTER END LOW TONE  cat=Sk  count=7
'*'  U+002A  ASTERISK  cat=Po  count=7
'\ue03b'  U+E03B  ?  cat=Co  count=6
'>'  U+003E  GREATER-THAN SIGN  cat=Sm  count=6
'\ue049'  U+E049  ?  cat=Co  count=5
'\ue03a'  U+E03A  ?  cat=Co  count=5
'\ue039'  U+E039  ?  cat=Co  count=5
'’'  U+2019  RIGHT SINGLE QUOTATION MARK  cat=Pf  count=4
'ˀ'  U+02C0  MODIFIER LETTER GLOTTAL STOP  cat=Lm  count=4
'ˁ' 

In [6]:
import re
import unicodedata
from langchain_core.documents import Document

MAP_GLOTTAL_TO_ARABIC = True
CHAR_FIXES = {
    "ي": "ی",
    "ك": "ک",
    "ة": "ه",
    "ۀ": "ه",
    "أ": "ا",
    "إ": "ا",
    "ٱ": "ا",
    "ؤ": "و",
    "ﻻ": "لا",
    "ͷ": "ک",
    "ͅ": "ی",
    "ͬ": "ی",
    "ͽ": "،",
    "“": "«",
    "”": "»",
    "‘": "'",
    "’": "'",
    "‐": "-",
    "\u0379": "",
    "\u02fc": "",
}
if MAP_GLOTTAL_TO_ARABIC:
    CHAR_FIXES.update(
        {
            "\u02bf": "ع",
            "\u02c1": "ع",
            "\u02c0": "ء",
        }
    )
else:
    CHAR_FIXES.update(
        {
            "\u02bf": "",
            "\u02c1": "",
            "\u02c0": "",
        }
    )
WS_EQUIVS = {
    "\u00a0": " ",
    "\u202f": " ",
    "\u2000": " ",
    "\u2001": " ",
    "\u2002": " ",
    "\u2003": " ",
    "\u2004": " ",
    "\u2005": " ",
    "\u2006": " ",
    "\u2007": " ",
    "\u2008": " ",
    "\u2009": " ",
    "\u200a": " ",
    "\u2060": " ",
    "\u00ad": "",
}
FORMAT_CHARS = "".join(
    [
        "\u200e",
        "\u200f",
        "\u202a",
        "\u202b",
        "\u202d",
        "\u202e",
        "\u202c",
        "\u2066",
        "\u2067",
        "\u2068",
        "\u2069",
        "\ufeff",
    ]
)
PUA_BULLETS = {"\ue039", "\ue03a", "\ue03b", "\ue03c", "\ue049"}
PERSIAN_DIACRITICS = re.compile(r"[\u064B-\u065F\u0670\u06D6-\u06ED]")


def normalize_pua(text: str) -> str:
    text = re.sub(r"(?m)^[ \t]*[" + "".join(PUA_BULLETS) + r"][ \t]*", "• ", text)
    for pua in PUA_BULLETS:
        text = text.replace(pua, " - ")
    return text


def remove_unassigned(text: str) -> str:
    return "".join(ch for ch in text if unicodedata.category(ch) != "Cn")


def normalize_pdf_text_advanced(text: str) -> str:
    text = unicodedata.normalize("NFKC", text)
    for k, v in WS_EQUIVS.items():
        text = text.replace(k, v)
    text = text.translate({ord(ch): None for ch in FORMAT_CHARS})
    for bad, good in CHAR_FIXES.items():
        text = text.replace(bad, good)
    text = normalize_pua(text)
    text = remove_unassigned(text)
    text = re.sub(r"(?m)^\s*\*\s+", "• ", text)
    text = re.sub(r"(?<=\s)&(?=\s)", " و ", text)
    text = PERSIAN_DIACRITICS.sub("", text)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\s+([،,:;!؟.])", r"\1", text)
    text = re.sub(r"([،,:;!؟.])([^\s\n])", r"\1 \2", text)
    text = re.sub(r"(?<![.!؟:؛\-])\n(?!\n)", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


In [7]:
clean_docs = [
    Document(
        page_content=normalize_pdf_text_advanced(d.page_content), metadata=d.metadata
    )
    for d in pdf_docs
]

print(clean_docs[min(15, len(clean_docs) - 1)].page_content[:1200])


تولد یک نرد، بخش دوم صفحه ١١ و او می خواست من را هم در این تجربه شریک کند. همچنین می خواست من را به ریاضی علاقمند کند.
پس من را روی زانو هایش می نشاند و از من می خواست تا برنامه هایی که با دقت روی کاغذ نوشته بود را برایش تایپ کنم. می گفت خودش با کامپیوترها راحت نیست. نمی دانم آن محاسبات راجع به چه چیزی بودند و بعید می دانم که آن موقع هیچ درکی از کاری که می کردم هم داشته باشم ولی به هرحال آنجا بودم و به او کمک می کردم. احتمالا کار از حالتی که او خودش به تنهایی برنامه ها را وارد می کرد، خیلی بیشتر طول می کشید. ولی کسی چه می داند؟ من از همان کودکی به صفحه کلید عادت کرده بودم، چیزی که پدربزرگم هیچ وقت ام یان اش را نداشت. بعد از مدرسه یا هر موقع دی، ری که مادرم من را پیش پدربزرگم می گذاشت، مشغول همین کار می شدیم.
بعد شروع کردم به خواندن راهنماهای کامپیوتر و وارد کردن برنامه های آماده شده. مثال ها شامل بازی های ساده ای بودند که خودتان می توانستید آن ها را وارد کنید. اگر همه چیز را درست تایپ می کردید، یک آقایی با گرافیک بد، روی صفحه راه می رفت. بعد می توانستید برنامه را عوض کنید تا آقای راه ر

In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://linuxbook.ir/all.html",),
    header_template={"User-Agent": "Mozilla/5.0"},
)
loader = WebBaseLoader("https://linuxbook.ir/all.html")
web_docs = loader.load()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
print(web_docs[0].page_content[:512])








لینوکس و زندگی | لینوکس و زندگی










Toggle navigation




لینوکس و زندگی



روی جلد
درباره
دانلود
حمایت













لینوکس و زندگی


فهرست 
لینوکس و زندگی. نوشته جادی www.jadi.netبرای دسترسی به آخرین نسخه کتاب در فرمت‌های مختلف و همچنین حمایت مالی از پروژه کتاب‌هایی برای گیک‌ها، به سایت www.jadi.ir و بخش حمایت مراجعه کنید.درباره این کتاب 
این کتاب سعی می کنه به خواننده ایده‌هایی درمورد زندگی و لینوکس بده. چرا زندگی؟ چون لینوکس یک فلسفه است و براومده از یک جامعه و کسی که می خواد توش موفق باشه با


In [10]:
from langchain_community.document_loaders import WikipediaLoader
import time, random

wiki_titles = [
    "ریچارد استالمن",
    "لینوس توروالدز",
    "لینوکس",
    "پروژه گنو",
    "نرم‌افزار آزاد",
    "بنیاد نرم‌افزار آزاد",
]


def load_wiki_title(title: str, retries: int = 3):
    last_err = None
    for attempt in range(1, retries + 1):
        try:
            loader = WikipediaLoader(query=title, lang="fa", load_max_docs=1)
            docs = loader.load()
            if docs:
                return docs[0]
        except Exception as e:
            last_err = e
        time.sleep(random.uniform(2.0, 5.0) + 0.5 * attempt)
    try:
        loader_en = WikipediaLoader(query=title, lang="en", load_max_docs=1)
        docs_en = loader_en.load()
        if docs_en:
            return docs_en[0]
    except Exception as e:
        last_err = e
    raise RuntimeError(f"Failed to load Wikipedia page for '{title}'") from last_err


wiki_docs = []
for t in wiki_titles:
    try:
        doc = load_wiki_title(t, retries=3)
        wiki_docs.append(doc)
        time.sleep(random.uniform(1.0, 2.5))
    except Exception as e:
        print(f"Warning: {t} failed: {e}")


In [11]:
print("Number of wikipedia pages (loaded Document objects):", len(wiki_docs))


Number of wikipedia pages (loaded Document objects): 6


In [12]:
from langchain_community.document_loaders import DirectoryLoader, BSHTMLLoader
from pathlib import Path

HTML_DIR = Path("data/html")
if not HTML_DIR.exists():
    HTML_DIR = Path("html")
if not HTML_DIR.exists():
    raise FileNotFoundError(f"HTML directory not found at: {HTML_DIR.resolve()}")
loader = DirectoryLoader(
    str(HTML_DIR),
    glob="**/*.html",
    loader_cls=BSHTMLLoader,
    loader_kwargs={"open_encoding": "utf-8"},
    show_progress=True,
    use_multithreading=True,
)
html_docs = loader.load()


 27%|███████████▏                             | 49/179 [00:00<00:00, 277.70it/s]/home/alaa/miniconda3/envs/quera/lib/python3.11/site-packages/langchain_community/document_loaders/html_bs.py:126: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(f, **self.bs_kwargs)
100%|█████████████████████████████████

In [13]:
print("Number of pages (loaded Document objects):", len(html_docs))


Number of pages (loaded Document objects): 179


In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

pdf_like = locals().get("clean_docs") or locals().get("pdf_docs")
web_like = locals().get("web_docs")
wiki_like = locals().get("wiki_docs")
html_like = locals().get("html_docs")
all_docs = []
for group in (pdf_like, web_like, wiki_like, html_like):
    if group:
        all_docs.extend(group)
if not all_docs:
    raise RuntimeError(
        "No documents found. Make sure you ran the previous loading cells successfully."
    )
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=150,
    separators=["\n\n", "\n", "؛", "؟", ".", "!", "،", " ", ""],
    add_start_index=True,
    length_function=len,
)
splitted_docs = [
    d for d in splitter.split_documents(all_docs) if d.page_content.strip()
]
print("The number of splitted documents:", len(splitted_docs))


The number of splitted documents: 2652


In [15]:
import os, getpass
from langchain_cohere import CohereEmbeddings

os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY") or getpass.getpass("Enter COHERE_API_KEY: ")

embeddings = CohereEmbeddings(
    model="embed-multilingual-v3.0",
    cohere_api_key=os.environ["COHERE_API_KEY"],
)


Enter COHERE_API_KEY:  ········


In [16]:
from langchain_chroma import Chroma
from pathlib import Path
import time, random, math

if not locals().get("splitted_docs"):
    raise RuntimeError("splitted_docs not found. Run the splitting cell first.")
PERSIST_DIR = Path("./collections/llm_corpus")
PERSIST_DIR.mkdir(parents=True, exist_ok=True)
try:
    import chromadb

    client = chromadb.PersistentClient(path=str(PERSIST_DIR))
except Exception as e:
    raise RuntimeError(
        "chromadb is required for persistence. Try: pip install chromadb"
    ) from e
vectorstore = Chroma(
    client=client,
    collection_name="llm_corpus",
    embedding_function=embeddings,
)


def maybe_persist(vs):
    try:
        cl = getattr(vs, "_client", None)
        if cl and hasattr(cl, "persist"):
            cl.persist()
    except Exception:
        pass


def add_in_batches(
    vs,
    documents,
    batch_size: int = 8,
    base_sleep: float = 3.0,
    polite_delay: tuple = (1.5, 3.5),
    max_retries: int = 6,
):
    try:
        from cohere.errors import TooManyRequestsError as Cohere429
    except Exception:

        class Cohere429(Exception): ...

    total = len(documents)
    for start in range(0, total, batch_size):
        batch = documents[start : start + batch_size]
        for attempt in range(1, max_retries + 1):
            try:
                vs.add_documents(batch)
                if ((start // batch_size) % 5) == 0:
                    maybe_persist(vs)
                break
            except Exception as e:
                msg = str(e)
                is_rate = (
                    isinstance(e, Cohere429)
                    or "429" in msg
                    or "TooManyRequests" in msg
                    or "rate limit" in msg.lower()
                )
                if is_rate and attempt < max_retries:
                    sleep_s = base_sleep * (2 ** (attempt - 1)) + random.uniform(
                        0.0, 1.2
                    )
                    print(
                        f"[429] Backoff {sleep_s:.1f}s | batch {start//batch_size+1}/{math.ceil(total/batch_size)} | try {attempt}/{max_retries}"
                    )
                    time.sleep(sleep_s)
                    continue
                raise
        time.sleep(random.uniform(*polite_delay))
    maybe_persist(vs)


add_in_batches(vectorstore, splitted_docs, batch_size=8, base_sleep=3.0, max_retries=6)
try:
    count = vectorstore._collection.count()
except Exception:
    count = len(vectorstore.get()["ids"])
print(f"Vectorstore ready. Items: {count}")


Vectorstore ready. Items: 2652


In [17]:
from langchain_cohere import ChatCohere
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

if "vectorstore" not in globals():
    raise RuntimeError(
        "vectorstore not found. Please build embeddings & Chroma vector store first."
    )
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 6, "fetch_k": 24, "lambda_mult": 0.5},
)


def format_docs_for_prompt(docs, max_chars=1400):
    chunks = []
    for d in docs:
        txt = d.page_content
        if len(txt) > max_chars:
            txt = txt[:max_chars]
        src = (
            d.metadata.get("source")
            or d.metadata.get("url")
            or d.metadata.get("file_path")
            or d.metadata.get("path")
        )
        if src:
            chunks.append(f"[منبع: {src}]\n{txt}")
        else:
            chunks.append(txt)
    return "\n\n-----\n\n".join(chunks)


def top_unique_sources(docs, limit=5):
    out, seen = [], set()
    for d in docs:
        src = (
            d.metadata.get("source")
            or d.metadata.get("url")
            or d.metadata.get("file_path")
            or d.metadata.get("path")
        )
        if not src or src in seen:
            continue
        seen.add(src)
        out.append(
            {
                "source": src,
                "title": d.metadata.get("title"),
                "page": d.metadata.get("page") or d.metadata.get("page_number"),
                "snippet": d.page_content[:320],
            }
        )
        if len(out) >= limit:
            break
    return out


SYSTEM = (
    "تو یک دستیار RAG فارسی هستی. با استفاده از متن‌های بازیابی‌شده، "
    "پاسخی روشن، دقیق و کامل بنویس. اگر مفید است از بولت‌پوینت استفاده کن. "
    "در پایان یک بخش «منابع» بیاور و ۳–۵ مرجع برتر را فهرست کن."
)
PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM),
        (
            "human",
            "سوال: {question}\n\n"
            "متن‌های بازیابی‌شده:\n{context}\n\n"
            "پاسخ کامل و دقیق بده و در پایان «منابع» را لیست کن.",
        ),
    ]
)
llm = ChatCohere(model="command-r-08-2024", temperature=0.2)
core_chain = (
    {
        "question": RunnablePassthrough(),
        "context": RunnablePassthrough()
        | retriever
        | RunnableLambda(format_docs_for_prompt),
    }
    | PROMPT
    | llm
    | StrOutputParser()
)


def ask(question: str):
    """High-level RAG function.
    Returns: {'answer': <rich text>, 'sources': [ {source,title,page,snippet}, ... ]}
    """
    docs = retriever.get_relevant_documents(question)
    answer_text = core_chain.invoke(question)
    return {
        "answer": answer_text.strip(),
        "sources": top_unique_sources(docs, limit=5),
    }


print("Full‑Power RAG is ready. Use: res = ask('سوال شما چیست؟'); print(res['answer'])")


Full‑Power RAG is ready. Use: res = ask('سوال شما چیست؟'); print(res['answer'])


In [18]:
res = ask("بهترین زبان برنامه نویسی چیه؟")
print(res["answer"])
for i, s in enumerate(res["sources"], 1):
    print(f"[{i}] {s['source']}")


/tmp/ipykernel_547613/474562247.py:95: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


بهترین زبان برنامه‌نویسی به عوامل مختلفی بستگی دارد، از جمله اهداف، زمینه کاری، و ترجیحات شخصی. با این حال، برخی زبان‌ها به دلیل قدرت، انعطاف‌پذیری، و کاربرد گسترده‌شان، به عنوان گزینه‌های برتر شناخته می‌شوند.

- **Python:** پایتون یکی از محبوب‌ترین و پرکاربردترین زبان‌های برنامه‌نویسی است. ساده، خوانا، و دارای کتابخانه‌های گسترده‌ای است که آن را برای توسعه سریع نرم‌افزار مناسب می‌کند. پایتون در زمینه‌های مختلف، از هوش مصنوعی و یادگیری ماشین گرفته تا توسعه وب و تحلیل داده، کاربرد دارد.

- **JavaScript:** جاوااسکریپت زبان اصلی وب است و برای توسعه تعاملی و پویا در مرورگرها استفاده می‌شود. با گسترش فناوری‌های وب، جاوااسکریپت به یک زبان قدرتمند و انعطاف‌پذیر تبدیل شده است که در توسعه سمت کلاینت و سمت سرور کاربرد دارد.

- **Java:** جاوا زبان برنامه‌نویسی شیءگرا است که در زمینه‌های مختلف، از توسعه نرم‌افزار گرفته تا برنامه‌های موبایل و سیستم‌های سازمانی، کاربرد دارد. جاوا دارای کتابخانه‌های گسترده و جامعه توسعه‌دهندگان فعال است.

- **C++:** سی‌پلاس‌پلاس زبان قدرتمندی است که برای توسعه نرم‌اف